# 線形計画法

- シンプレックス法

In [40]:
# display graph on notebook
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

In [104]:
"""
最大化する目的関数
z = 2x1 + 3x2

制約条件
x1 + 2x2 <= 14
x1 + x2 <= 8
3x1 + x2 <= 18

解
x1 = 2
x2 = 6
z = 22
"""
objective = np.array([2., 3.])
constraints = np.array(
    [
        [1., 2., 14],
        [1., 1., 8],
        [3., 1., 18]
    ]
)

In [128]:
def simplex_lp(objective, constraints):
    """
    Parameters
    ----------
    objective : numpy.array
        最大化したい目的関数の係数ベクトル (n次元)
        c1 x1 + c2 x2 + ... + cn xn

    constraints : numpy.array
        制約条件の係数行列 (m行n+1列)
        a11 x1 + a12 x2 + ... + a1n xn <= b1
        ...
        am1 x1 + am2 x2 + ... + amn xn <= bm
    """
    # 処理しやすいように整形
    m = constraints.shape[0]
    n = constraints.shape[1] - 1
    a = constraints[:,0:n]
    b = constraints[:,-1]
    new_constraints = np.concatenate((a, np.identity(m), np.reshape(b, (m, 1))), axis=1)
    new_objective = np.concatenate((objective * -1, [0]*(m+1)), axis=0).reshape(1,n+m+1)
    matrix = np.concatenate((new_constraints, new_objective), axis=0)
    print(matrix)
    

    # 実行可能解を持つかどうかの判定
    if np.any(b < 0):
        # 負のbを持つ場合は補助問題を先に解く必要がある
        print('ダメ')

    
    try_num = 0
    try_num_max = 100
    while try_num < try_num_max:
        try_num += 1
    
        # 変数 x_i を増やせば目的関数が大きくなる係数 c_i を見つける
        i_c_min = 0
        c_min = matrix[m][i_c_min]
        for i in range(n + m):
            if matrix[m][i] < c_min:
                i_c_min = i
                c_min = matrix[m][i_c_min]
        if c_min >= 0:
            # どの x_i を増やしても目的関数は増大しない状態なので処理終了
            break
    
        # x_i_c_min を増やしていったときに一番早く制約の上限に達するものを見つける
        i_fastest_limit = 0
        fastest_limit = matrix[i_fastest_limit][-1] / matrix[i_fastest_limit][i_c_min]
        for i in range(m):
            limit = matrix[i][-1] / matrix[i][i_c_min]
            if limit < fastest_limit:
                i_fastest_limit = i
                fastest_limit = limit
    
        matrix[i_fastest_limit] /= matrix[i_fastest_limit][i_c_min]
        for i in range(m+1):
            if i != i_fastest_limit:
                matrix[i] -= matrix[i_fastest_limit] * matrix[i][i_c_min]
    
        print('Trial {}:'.format(try_num))
        print(matrix)
    return matrix


simplex_lp(objective, constraints)

[[ 1.  2.  1.  0.  0. 14.]
 [ 1.  1.  0.  1.  0.  8.]
 [ 3.  1.  0.  0.  1. 18.]
 [-2. -3.  0.  0.  0.  0.]]
Trial 1:
[[ 0.5  1.   0.5  0.   0.   7. ]
 [ 0.5  0.  -0.5  1.   0.   1. ]
 [ 2.5  0.  -0.5  0.   1.  11. ]
 [-0.5  0.   1.5  0.   0.  21. ]]
Trial 2:
[[ 0.  1.  1. -1.  0.  6.]
 [ 1.  0. -1.  2.  0.  2.]
 [ 0.  0.  2. -5.  1.  6.]
 [ 0.  0.  1.  1.  0. 22.]]


array([[ 0.,  1.,  1., -1.,  0.,  6.],
       [ 1.,  0., -1.,  2.,  0.,  2.],
       [ 0.,  0.,  2., -5.,  1.,  6.],
       [ 0.,  0.,  1.,  1.,  0., 22.]])